In [1]:
import sys
!{sys.executable} -m pip install pandas numpy matplotlib seaborn scikit-learn xgboost imbalanced-learn openpyxl ftfy deep-translator



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel('Triage_Dataset.xlsx')

In [3]:
# Load dataset and do initial inspection
print('Loaded dataset with shape:', df.shape)

display(df.head())
print('Dtypes:', df.dtypes)

print("\nColumns:") 
print(df.columns.tolist())

print('Numeric summary:')
display(df.describe(include=[np.number]).transpose())

print('Missing values per column:')
print(df.isna().sum())

print('Duplicate rows:', df.duplicated().sum())

Loaded dataset with shape: (1267, 24)


,Group,Sex,Age,Patients number per hour,Arrival mode,Injury,Chief_complain,Mental,Pain,NRS_pain,...,BT,Saturation,KTAS_RN,Diagnosis in ED,Disposition,KTAS_expert,Error_group,Length of stay_min,KTAS duration_min,mistriage
0,2,2,71.0,3,3,2,right ocular pain,1,1,2.0,...,36.6,100,2,Corneal abrasion,1,4,2,86.0,5.000000,1
1,1,1,56.0,12,3,2,right forearm burn,1,1,2.0,...,36.5,NaN,4,"Burn of hand, firts degree dorsum",1,5,4,64.0,3.950000,1
2,2,1,67.5,8,2,2,"arm pain, Lt",1,1,2.0,...,36.6,98,4,"Fracture of surgical neck of humerus, closed",2,5,4,862.0,1.000000,1
3,1,2,71.0,8,1,1,ascites tapping,1,1,3.0,...,36.5,NaN,4,Alcoholic liver cirrhosis with ascites,1,5,6,108.0,9.833333,1
4,1,2,58.0,4,3,1,"distension, abd",1,1,3.0,...,36.5,NaN,4,Ascites,1,5,8,109.0,6.600000,1


Dtypes: Group                         int64
Sex                           int64
Age                         float64
Patients number per hour      int64
Arrival mode                  int64
Injury                        int64
Chief_complain               object
Mental                        int64
Pain                          int64
NRS_pain                    float64
SBP                          object
DBP                          object
HR                           object
RR                           object
BT                           object
Saturation                   object
KTAS_RN                       int64
Diagnosis in ED              object
Disposition                   int64
KTAS_expert                   int64
Error_group                   int64
Length of stay_min          float64
KTAS duration_min           float64
mistriage                     int64
dtype: object

Columns:
['Group', 'Sex', 'Age', 'Patients number per hour', 'Arrival mode', 'Injury', 'Chief_complain', 'Mental'

,count,mean,std,min,25%,50%,75%,max
Group,1267.0,1.456985,0.498343,1.000000,1.0,1.000000,2.0,2.00000
Sex,1267.0,1.521705,0.499726,1.000000,1.0,2.000000,2.0,2.00000
Age,1267.0,54.394396,19.722260,15.500000,37.0,57.000000,71.0,95.80000
Patients number per hour,1267.0,7.519337,3.160563,1.000000,5.0,7.000000,10.0,17.00000
Arrival mode,1267.0,2.820837,0.807904,1.000000,2.0,3.000000,3.0,7.00000
Injury,1267.0,1.192581,0.394482,1.000000,1.0,1.000000,1.0,2.00000
Mental,1267.0,1.105762,0.447768,1.000000,1.0,1.000000,1.0,4.00000
Pain,1267.0,0.563536,0.496143,0.000000,0.0,1.000000,1.0,1.00000
NRS_pain,711.0,4.104079,1.419332,1.000000,3.0,4.000000,5.0,10.00000
KTAS_RN,1267.0,3.335438,0.885391,1.000000,3.0,3.000000,4.0,5.00000


Missing values per column:
Group                         0
Sex                           0
Age                           0
Patients number per hour      0
Arrival mode                  0
Injury                        0
Chief_complain                0
Mental                        0
Pain                          0
NRS_pain                    556
SBP                           0
DBP                           0
HR                            0
RR                            0
BT                            0
Saturation                  688
KTAS_RN                       0
Diagnosis in ED               2
Disposition                   0
KTAS_expert                   0
Error_group                   0
Length of stay_min            0
KTAS duration_min             0
mistriage                     0
dtype: int64
Duplicate rows: 0


In [4]:
import re

hangul_re = re.compile(r'[\uac00-\ud7a3]')

df_clean = df.copy()

mask = df_clean.astype(str).apply(lambda col: col.str.contains(hangul_re, na=False)).any(axis=1)
count = mask.sum()

if count > 0:
    hangul_df = df_clean[mask].copy()
    hangul_df.to_excel('rows_with_hangul.xlsx', index=False)
    df_clean = df_clean[~mask].reset_index(drop=True)
    df_clean.to_excel('cleaned.xlsx', index=False)
    print(f'Found {count} rows with Hangul. Files saved.')
else:
    print('No Hangul detected; cleaned file unchanged.')


Found 58 rows with Hangul. Files saved.


In [ ]:
import numpy as np

df_clean = pd.read_excel('cleaned.xlsx')

num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
for c in num_cols:
    med = df_clean[c].median()
    df_clean[c] = df_clean[c].fillna(med)
    nonnull = df_clean[c].dropna()
    try:
        if not nonnull.empty and np.all(np.equal(np.mod(nonnull.astype(float), 1), 0)):
            df_clean[c] = df_clean[c].astype('Int64')
    except Exception:
        pass

obj_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()
for c in obj_cols:
    if df_clean[c].isna().any():
        modes = df_clean[c].mode(dropna=True)
        if not modes.empty:
            df_clean[c] = df_clean[c].fillna(modes[0])

int_cols = ["Age", "NRS_pain", "SBP", "DBP", "RR", "Saturation"]
for col in int_cols:
    df_clean[col] = df_clean[col].astype(str)
    df_clean[col] = df_clean[col].str.replace(r'[^0-9\-]', '', regex=True)
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    df_clean[col] = df_clean[col].astype("Int64")
    if col == "Age":
        df_clean[col] = df_clean[col].round()

    df_clean[col] = df_clean[col].astype("Int64")

df_clean["BT"] = (
    df_clean["BT"]
    .astype(str)
    .str.replace(r'[^0-9.\-]', '', regex=True)
    .replace("", np.nan)
    .astype(float)
)

print('Dtypes after reading cleaned file:')
print(df_clean.dtypes)

df_clean.to_excel('cleaned.xlsx', index=False)
print('Saved cleaned file to cleaned.xlsx')


Dtypes after reading cleaned file:
Group                         Int64
Sex                           Int64
Age                           Int64
Patients number per hour      Int64
Arrival mode                  Int64
Injury                        Int64
Chief_complain               object
Mental                        Int64
Pain                          Int64
NRS_pain                      Int64
SBP                           Int64
DBP                           Int64
HR                            Int64
RR                            Int64
BT                          float64
Saturation                    Int64
KTAS_RN                       Int64
Diagnosis in ED              object
Disposition                   Int64
KTAS_expert                   Int64
Error_group                   Int64
Length of stay_min          float64
KTAS duration_min           float64
mistriage                     Int64
dtype: object
Saved cleaned file to cleaned_minimal.xlsx
